# Imports and Constants

An API key must be given. Registration required first.

In [2]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil import parser


MY_API_KEY = ''

# API-Call functions

TODO: These could probably be placed in a utils module?

In [3]:
def get_album_matches(artist, album):
    base_str = 'http://ws.audioscrobbler.com/2.0/?'
    method_str = 'method=album.search'
    artist_str = '&artist='+artist
    album_str = '&album='+album
    format_str = '&format=json'
    api_str = '&api_key=' + MY_API_KEY

    request_str = base_str + method_str + artist_str + album_str + api_str + format_str
    return requests.get(request_str).json()

In [4]:
def get_album_info(artist, album):    
    base_str = 'http://ws.audioscrobbler.com/2.0/?'
    method_str = 'method=album.getinfo'
    artist_str = '&artist='+str(artist).strip().replace('&','%26')
    album_str = '&album='+str(album).strip().replace('&','%26')
    format_str = '&format=json'
    api_str = '&api_key=' + MY_API_KEY

    request_str = base_str + method_str + artist_str + album_str + api_str + format_str
    try:
        r_data = requests.get(request_str).json()['album']
    except KeyError:
        r_data = np.nan

    return r_data

In [5]:
def get_track_info(artist, album, track):    
    base_str = 'http://ws.audioscrobbler.com/2.0/?'
    method_str = 'method=track.getinfo'
    artist_str = '&artist='+str(artist).strip().replace('&','%26')
    album_str = '&album='+str(album).strip().replace('&','%26')
    track_str = '&track='+str(track).strip().replace('&', '%26')
    format_str = '&format=json'
    api_str = '&api_key=' + MY_API_KEY

    request_str = base_str + method_str + artist_str + album_str + track_str + api_str + format_str
    # print(request_str)
    try:
        r_data = requests.get(request_str).json()['track']
    except KeyError:
        r_data = np.nan

    return r_data

In [6]:
def get_release_date(row_info):
    try:
        release_date = row_info['releasedate']
    except KeyError:
        release_date = '1 Jan 0001, 00:00'
    return datetime.strptime(release_date, '%d %b %Y, %H:%M')


# Data analysis

Show the first 5 entries of the Excel-file.

In [9]:
df = pd.read_excel('04 History of Metal.xlsx', sheet_name='Gesamt')
df = df.drop(labels=['Unnamed: 9', 0.5, 'Year', 'Month', 'Day', 'Spotify', 'Factor', 'Score'], axis=1)
df = df.dropna(axis=0)
df.head()

,Number,Band,Album
0,1.0,Black Sabbath,Paranoid
1,2.0,Led Zeppelin,Led Zeppelin IV
2,3.0,Deep Purple,Machine Head
3,4.0,Queen,A Night at the Opera
4,5.0,Queen,Jazz


Many albums occur several times with similar names. Their counts should be summed up.
See the following example of the top 5 albums.

In [12]:
results_df = pd.DataFrame()

for idx, row in df.head().iterrows():
    artist = row['Band']
    album = row['Album']
    matches = get_album_matches(artist, album)

    for match in matches['results']['albummatches'].values():
        for d in match:
            if d['artist'] == artist:
                results_df = results_df.append({
                    'artist': artist,
                    'album': album,
                    'album_instance': d['name'],
                    'lastfm_info': get_album_info(d['artist'], d['name'])
                }, ignore_index=True)
                print(d['artist'], '-', d['name'])
                # print(results_df.size)

Black Sabbath - Paranoid
Black Sabbath - Paranoid (2009 Remastered Version)
Black Sabbath - Paranoid (Remastered)
Black Sabbath - Paranoid (2014 Remaster)
Black Sabbath - Paranoid (Remaster)
Black Sabbath - Paranoid (Remastered Edition)
Black Sabbath - Paranoid (Deluxe Edition)
Black Sabbath - Paranoid (Deluxe Expanded Edit
Black Sabbath - Paranoid (Original Album) Disc 1
Black Sabbath - Paranoid (Deluxe Expanded Edition) CD1
Black Sabbath - Paranoid (Black Box: Disc 2)
Black Sabbath - Paranoid (Us 1st Press, Wbm 3104-2)
Black Sabbath - Paranoid (Deluxe Expanded Edition, 2cd, Sanctuary 1782444)
Black Sabbath - Paranoid [2009, Sanctuary, 1782444]
Black Sabbath - Paranoid - Black Box -  The Complete Original Black Sabbath
Black Sabbath - Paranoid [1989, Vertigo, 23PD-134]
Black Sabbath - Paranoid [Deluxe Edition]
Black Sabbath - Paranoid (Black Box, R2 73923-B)
Led Zeppelin - Led Zeppelin IV
Led Zeppelin - Led Zeppelin IV (Deluxe Edition)
Led Zeppelin - Led Zeppelin IV (1994 Remaster
Led

Add some entries of the lastfm info as column to the dataframe.

In [15]:
df = results_df
df['listeners'] = df.apply(lambda row: int(row['lastfm_info']['listeners']), axis=1)
df['playcount'] = df.apply(lambda row: int(row['lastfm_info']['playcount']), axis=1)
df['release'] = df.apply(lambda row: get_release_date(row['lastfm_info']), axis=1)
df['url'] = df.apply(lambda row: row['lastfm_info']['url'], axis=1)
df

,album,album_instance,artist,lastfm_info,listeners,playcount,release,url
0,Paranoid,Paranoid,Black Sabbath,"{'name': 'Paranoid', 'artist': 'Black Sabbath'...",1259686,15301016,0001-01-01 00:00:00,https://www.last.fm/music/Black+Sabbath/Paranoid
1,Paranoid,Paranoid (2009 Remastered Version),Black Sabbath,"{'name': 'Paranoid (2009 Remastered Version)',...",253077,2833544,0001-01-01 00:00:00,https://www.last.fm/music/Black+Sabbath/Parano...
2,Paranoid,Paranoid (Remastered),Black Sabbath,"{'name': 'Paranoid (Remastered)', 'artist': 'B...",170138,1435213,0001-01-01 00:00:00,https://www.last.fm/music/Black+Sabbath/Parano...
3,Paranoid,Paranoid (2014 Remaster),Black Sabbath,"{'name': 'Paranoid (2014 Remaster)', 'artist':...",78860,543325,0001-01-01 00:00:00,https://www.last.fm/music/Black+Sabbath/Parano...
4,Paranoid,Paranoid (Remaster),Black Sabbath,"{'name': 'Paranoid (Remaster)', 'artist': 'Bla...",49691,272693,0001-01-01 00:00:00,https://www.last.fm/music/Black+Sabbath/Parano...
...,...,...,...,...,...,...,...,...
124,A Night at the Opera,(1975) A Night at the Opera,Queen,"{'name': '(1975) A Night at the Opera', 'artis...",252,5655,0001-01-01 00:00:00,https://www.last.fm/music/Queen/(1975)+A+Night...
125,Jazz,Jazz (2011 Remaster),Queen,"{'name': 'Jazz (2011 Remaster)', 'artist': 'Qu...",401655,3045202,0001-01-01 00:00:00,https://www.last.fm/music/Queen/Jazz+(2011+Rem...
126,Jazz,Jazz,Queen,"{'name': 'Jazz', 'artist': 'Queen', 'mbid': 'b...",396706,5128277,0001-01-01 00:00:00,https://www.last.fm/music/Queen/Jazz
127,Jazz,Jazz (Deluxe Remastered Version),Queen,"{'name': 'Jazz (Deluxe Remastered Version)', '...",239674,1309473,0001-01-01 00:00:00,https://www.last.fm/music/Queen/Jazz+(Deluxe+R...


Sum up the different entries that correspond to the same album:

In [16]:
cumulative_df = df.drop(['album_instance'], axis=1).groupby(['artist','album']).sum()
cumulative_df

listeners  playcount
artist        album                                     
Black Sabbath Paranoid                1935741   21604633
Deep Purple   Machine Head             642413    6239290
Led Zeppelin  Led Zeppelin IV         1895318   26749309
Queen         A Night at the Opera    1598926   16617807
              Jazz                    1085091    9772558

Get the album last fm album entries with the most listeners.

In [18]:
idx = df.groupby(['artist', 'album'])['listeners'].transform(max) == df['listeners']
top_albums_df = df[idx]
top_albums_df

,album,album_instance,artist,lastfm_info,listeners,playcount,release,url
0,Paranoid,Paranoid,Black Sabbath,"{'name': 'Paranoid', 'artist': 'Black Sabbath'...",1259686,15301016,0001-01-01 00:00:00,https://www.last.fm/music/Black+Sabbath/Paranoid
18,Led Zeppelin IV,Led Zeppelin IV,Led Zeppelin,"{'name': 'Led Zeppelin IV', 'artist': 'Led Zep...",1048867,17007869,0001-01-01 00:00:00,https://www.last.fm/music/Led+Zeppelin/Led+Zep...
64,Machine Head,Machine Head,Deep Purple,"{'name': 'Machine Head', 'artist': 'Deep Purpl...",343905,4011632,0001-01-01 00:00:00,https://www.last.fm/music/Deep+Purple/Machine+...
83,A Night at the Opera,A Night at the Opera,Queen,"{'name': 'A Night at the Opera', 'artist': 'Qu...",650003,9415926,0001-01-01 00:00:00,https://www.last.fm/music/Queen/A+Night+at+the...
125,Jazz,Jazz (2011 Remaster),Queen,"{'name': 'Jazz (2011 Remaster)', 'artist': 'Qu...",401655,3045202,0001-01-01 00:00:00,https://www.last.fm/music/Queen/Jazz+(2011+Rem...


Get listeners per song

In [25]:
tracks_df = pd.DataFrame()

for idx, row in top_albums_df.iterrows():
    artist = row['artist']
    album = row['album']
    album_instance = row['album_instance']
    tracks = row['lastfm_info']['tracks']['track']

    for track in tracks:
        lastfm_info = get_track_info(artist, album, track['name'])

        tracks_df = tracks_df.append({
            'artist': artist,
            'album': album,
            'track': track['name'],
            'lastfm_info': lastfm_info
        }, ignore_index=True)

In [26]:
tracks_df['listeners'] = tracks_df.apply(lambda row: int(row['lastfm_info']['listeners']), axis=1)
tracks_df['playcount'] = tracks_df.apply(lambda row: int(row['lastfm_info']['playcount']), axis=1)
tracks_df = tracks_df.drop(['lastfm_info'], axis=1)
tracks_df.groupby('album')['listeners']
# tracks_df.drop(['lastfm_info'], axis=1).groupby(['artist', 'album']).sort(by='listeners')

top_tracks_df  = tracks_df.groupby(['album']).apply(lambda x: x.sort_values(['listeners'], ascending=False))
top_tracks_df

album         artist  \
album                                                          
A Night at the Opera 33  A Night at the Opera          Queen   
                     26  A Night at the Opera          Queen   
                     32  A Night at the Opera          Queen   
                     34  A Night at the Opera          Queen   
                     27  A Night at the Opera          Queen   
                     25  A Night at the Opera          Queen   
                     24  A Night at the Opera          Queen   
                     30  A Night at the Opera          Queen   
                     28  A Night at the Opera          Queen   
                     31  A Night at the Opera          Queen   
                     29  A Night at the Opera          Queen   
                     23  A Night at the Opera          Queen   
Jazz                 46                 Jazz           Queen   
                     36                 Jazz           Queen   
                     38                 Jazz           Queen   
                     37                 Jazz           Queen   
                     35                 Jazz           Queen   
                     40                 Jazz           Queen   
                     39                 Jazz           Queen   
                     41                 Jazz           Queen   
                     42                 Jazz           Queen   
                     43                 Jazz           Queen   
                     47                 Jazz           Queen   
                     45                 Jazz           Queen   
                     44                 Jazz           Queen   
Led Zeppelin IV      11       Led Zeppelin IV   Led Zeppelin   
                     8        Led Zeppelin IV   Led Zeppelin   
                     9        Led Zeppelin IV   Led Zeppelin   
                     14       Led Zeppelin IV   Led Zeppelin   
                     15       Led Zeppelin IV   Led Zeppelin   
                     10       Led Zeppelin IV   Led Zeppelin   
                     12       Led Zeppelin IV   Led Zeppelin   
                     13       Led Zeppelin IV   Led Zeppelin   
Machine Head         20          Machine Head    Deep Purple   
                     16          Machine Head    Deep Purple   
                     22          Machine Head    Deep Purple   
                     21          Machine Head    Deep Purple   
                     19          Machine Head    Deep Purple   
                     18          Machine Head    Deep Purple   
                     17          Machine Head    Deep Purple   
Paranoid             1               Paranoid  Black Sabbath   
                     3               Paranoid  Black Sabbath   
                     0               Paranoid  Black Sabbath   
                     2               Paranoid  Black Sabbath   
                     4               Paranoid  Black Sabbath   
                     7               Paranoid  Black Sabbath   
                     5               Paranoid  Black Sabbath   
                     6               Paranoid  Black Sabbath   

                                                             track  listeners  \
album                                                                           
A Night at the Opera 33                          Bohemian Rhapsody    1433059   
                     26                      You're My Best Friend     669044   
                     32                            Love of My Life     305967   
                     34                         God Save the Queen     187341   
                     27                                        '39     185898   
                     25                    I'm in Love with My Car     185641   
                     24               Lazing on a Sunday Afternoon     177315   
                     30                         Seaside Rendezvous     149460   
                     28      

# Export functions

Export results to Excel sheet

In [28]:
cumulative_df.to_excel('history_of_metal_lastfm_albums_cumulative.xlsx', sheet_name='albums_(cumulative)')
df.to_excel('history_of_metal_lastfm_albums_raw.xlsx', sheet_name='albums_(raw)')
top_tracks_df.to_excel('history_of_metal_lastfm_tracks.xlsx', sheet_name='tracks')